In [1]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

documents[0]

{'text': 'El Eje 4 contribuirá al repoblamiento y mejoramiento urbano del Centro Histórico de Puebla, protegiendo su patrimonio histórico cultural.',
 'section': 'Vinculación a otros instrumentos municipales',
 'question': '¿Cómo contribuirá el Eje 4 a la protección del patrimonio histórico cultural del Centro Histórico?',
 'topic': 'PMD',
 'id': '335b2f94'}

In [2]:
import pandas as pd
df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[20]

{'question': '¿Cuál es la relación entre turismo y economía local?',
 'topic': 'PMD',
 'document': '21b7970e'}

In [3]:
doc_idx = {d['id']: d for d in documents}
doc_idx['335b2f94']['text']

'El Eje 4 contribuirá al repoblamiento y mejoramiento urbano del Centro Histórico de Puebla, protegiendo su patrimonio histórico cultural.'

In [4]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "topic": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "ch-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ch-questions'})

In [6]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████| 186/186 [00:21<00:00,  8.81it/s]


In [7]:
def elastic_search_knn_combined(vector, topic):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "topic": topic
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "topic": topic
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "topic", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [8]:
def vector_combined_knn(q):
    question = q['question']
    topic = q['topic']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, topic)

In [9]:
vector_combined_knn(dict(
    question='¿Cuál es la relación entre turismo y economía local?',
    topic='PMD'
))

[{'question': '¿Qué importancia tiene el turismo para el desarrollo económico de Puebla?',
  'topic': 'PMD',
  'section': 'Turismo',
  'text': 'El turismo representa una actividad de primera importancia para el desarrollo económico de Puebla.',
  'id': '21b7970e'},
 {'question': '¿Cómo contribuyen los reconocimientos de la UNESCO al turismo en Puebla?',
  'topic': 'PMD',
  'section': 'Turismo',
  'text': 'Los reconocimientos de la UNESCO otorgados al municipio en materia cultural constituyen una ventaja competitiva que fortalece su vocación turística.',
  'id': '3b870dae'},
 {'question': '¿Cuál fue la tasa media anual de crecimiento poblacional en la zona metropolitana en el último decenio?',
  'topic': 'PMD',
  'section': 'Densidad poblacional',
  'text': 'En el último decenio, la zona metropolitana registró su menor crecimiento poblacional con una tasa media anual de 0.9%. Esta situación contrasta con las altas tasas de crecimiento de municipios como Cuautlancingo, San Andrés Cholula